## This notebook runs a RF on the whole MIC data (only train set of course) and save the 50 best features ranked by feature_importances

In [1]:
import sys
sys.path.remove('/home/pataki/.ipython')
sys.path.remove('/home/pataki/.local/lib/python3.6/site-packages')

In [2]:
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_predict

%matplotlib inline

In [3]:
pd.__version__, np.__version__, sk.__version__

('0.25.1', '1.16.5', '0.21.2')

In [4]:
# need to set manually the location of these files
base = '/mnt/net/onco2/data0/users/pataki/AMR_pred/final_processed/'

metaDF      = pd.read_csv(f'{base}meta.tsv', sep='\t')

micDF       = pd.read_csv(f'{base}ciprofloxacin_MICs.csv', dtype={'measurement':float})
zoneDF      = pd.read_csv(f'{base}ciprofloxacin_zone_diameters.csv', dtype={'measurement':float})
micDF  = micDF.append(zoneDF)

resfinderDF = pd.read_csv(f'{base}resfinder_results.csv')
snpDF       = pd.read_csv(f'{base}SNP_matrix.tsv', sep='\t', low_memory=False)
# need 64 GB RAM

In [5]:
# add new features: if any qnr appears + if any qnrS appears
qnr  = [i for i in resfinderDF.columns.tolist() if 'qnr' in i]
qnrS = [i for i in resfinderDF.columns.tolist() if 'qnrS' in i]

resfinderDF['has_qnr']  = (resfinderDF[qnr].sum(1) > 0).astype('int')
resfinderDF['has_qnrS'] = (resfinderDF[qnrS].sum(1) > 0).astype('int')

In [6]:
mic_resfinderDF = pd.merge(micDF[['sample_alias', 'measurement_units', 'measurement']], 
                           resfinderDF, on='sample_alias', how='inner')

mic_resfinderDF = pd.merge(metaDF[['sample_alias', 'country']], 
                           mic_resfinderDF, on='sample_alias', how='inner')

allDF = pd.merge(mic_resfinderDF, snpDF, on='sample_alias', how='inner')

In [7]:
metaDF.shape, allDF.shape

((807, 8), (807, 837742))

we don't want to touch the test data, separate it!

In [8]:
trainDF = allDF[(~allDF.country.isnull()) & (allDF.measurement_units == 'mg/L')]
testDF  = allDF[(allDF.country.isnull())  & (allDF.measurement_units == 'mg/L')]

len(trainDF), len(testDF)

(438, 266)

In [9]:
trainDF.drop(['sample_alias', 'measurement_units'], axis=1, inplace=True)

/home/pataki/.conda/envs/AMR/lib/python3.6/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [10]:
trainDF.head()

,country,measurement,ARR-2_1_HQ141279,ARR-3_1_JF806499,ARR-3_4_FM207631,aac(3)-IIa_1_X51534,aac(3)-IIa_2_AY138987,aac(3)-IIa_3_X13543,aac(3)-IIa_4_L22613,aac(3)-IIa_5_FJ012882,...,CP009074.1_9656,CP009074.1_97,CP009074.1_9776,CP009074.1_9803,CP009074.1_9867,CP009074.1_9891,CP009074.1_9932,CP009074.1_994,CP009074.1_9950,CP009074.1_997
0,Denmark,0.015,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Denmark,0.015,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Denmark,0.015,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Denmark,0.015,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Denmark,0.015,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**save feature importances for each fold in a leave-one-country-out validation**

In [12]:
rf = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

importancesDF = pd.DataFrame()
predictions  = []
measurements = []

for i in pd.unique(trainDF.country.values):
    print(i)
    train_X = trainDF[trainDF.country != i]
    train_y = train_X.pop('measurement')
    train_X.pop('country')
    
    test_X = trainDF[trainDF.country == i]
    test_y = test_X.pop('measurement')
    test_X.pop('country')
    
    rf.fit(train_X, np.log2(train_y))
    impDF = pd.DataFrame({'imp':rf.feature_importances_, 'feat':train_X.columns})
    impDF['country'] = i
    importancesDF = importancesDF.append(impDF)
    
    predictions  = predictions + list(2**np.round(rf.predict(test_X)))
    measurements = measurements + list(test_y)

Denmark
Italy
United Kingdom
USA
Viet Nam


### Check feature importances for each fold and the aggregation

In [13]:
imp_pivot = pd.pivot_table(importancesDF[importancesDF.imp>0], 
                           index='feat', columns='country', values='imp', fill_value=0)
imp_pivot['sum'] = imp_pivot.sum(1)
imp_pivot.sort_values('sum', ascending=False).head(50).to_csv(f'{base}top50_important_features.csv')

In [14]:
imp_pivot.sort_values('sum', ascending=False).head(15).round(3)

country,Denmark,Italy,USA,United Kingdom,Viet Nam,sum
feat,,,,,,
CP009072.1_167295,0.555,0.567,0.570,0.558,0.010,2.260
CP009072.1_167306,0.114,0.148,0.107,0.141,0.690,1.200
CP009072.1_1589734,0.181,0.151,0.199,0.172,0.001,0.705
has_qnrS,0.042,0.039,0.019,0.037,0.004,0.140
qnrS1_1_AB187515,0.012,0.007,0.017,0.004,0.000,0.041
blaCTX-M-55_1_DQ810789,0.002,0.009,0.009,0.013,0.000,0.033
blaVIM-48_1_KY362199,0.004,0.001,0.016,0.002,0.000,0.022
CP009072.1_3517597,0.000,0.000,0.000,0.000,0.013,0.014
CP009072.1_1734215,0.000,0.000,0.000,0.000,0.008,0.009


**Save the top50 features to a smaller table for easier/lighter further ML training.**

In [16]:
top50_feat = imp_pivot.sort_values('sum', ascending=False).head(50).index.tolist()

In [17]:
allDF[['sample_alias', 'country', 'measurement', 'measurement_units'] 
      + top50_feat].to_csv(f'{base}merged_top50.csv', index=False)